In [89]:
import os
import sys
from glob import glob
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import torch
from torch.nn import functional as F
from model import load_model
from torchvision import models

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
ENSEMBLE_PATH = '../saved_models/kfold-ensemble-VanillaEfficientNet_20210407'

In [12]:
folds = glob(os.path.join(ENSEMBLE_PATH, '*'))

In [14]:
fold = folds[0]

In [33]:
model = models.resnet18()

In [31]:
model_params = glob(os.path.join(fold, '*'))
model_params.sort(key=lambda x: float(x.split('f1(')[-1].split(')')[0]), reverse=True)
model_params

['../saved_models/kfold-ensemble-VanillaEfficientNet_20210407/fold01/Fold01_VanillaEfficientNet_task(main)ep(28)f1(0.8311)bs(32)loss(0.0014)lr(0.001)trans(random)optim(adam)crit(labelsmoothingLoss)seed(42).pth',
 '../saved_models/kfold-ensemble-VanillaEfficientNet_20210407/fold01/Fold01_VanillaEfficientNet_task(main)ep(17)f1(0.8257)bs(32)loss(0.0014)lr(0.001)trans(random)optim(adam)crit(labelsmoothingLoss)seed(42).pth',
 '../saved_models/kfold-ensemble-VanillaEfficientNet_20210407/fold01/Fold01_VanillaEfficientNet_task(main)ep(13)f1(0.8207)bs(32)loss(0.0014)lr(0.001)trans(random)optim(adam)crit(labelsmoothingLoss)seed(42).pth',
 '../saved_models/kfold-ensemble-VanillaEfficientNet_20210407/fold01/Fold01_VanillaEfficientNet_task(main)ep(10)f1(0.8006)bs(32)loss(0.0014)lr(0.001)trans(random)optim(adam)crit(labelsmoothingLoss)seed(42).pth',
 '../saved_models/kfold-ensemble-VanillaEfficientNet_20210407/fold01/Fold01_VanillaEfficientNet_task(main)ep(09)f1(0.7830)bs(32)loss(0.0014)lr(0.001)tra

In [147]:
fname = os.path.basename(model_params[0])
weight = float(fname.split('f1(')[-1].split(')')[0])

0.8311

0.8311

In [37]:
sample = torch.rand(2, 3, 224, 224)

In [62]:
preds_hard = []
preds_soft = []

In [52]:
output = model(sample)

In [56]:
soft = F.softmax(output, dim=1)

In [69]:
preds_hard.append(torch.argmax(soft, dim=1).data.numpy())
preds_soft.append(soft.data.numpy())
len(preds_hard), len(preds_soft)

(5, 5)

In [78]:
len(preds_soft), preds_soft[0].shape

(5, (2, 1000))

In [91]:
ids = [i for i in range(10)]

In [94]:
id_df = pd.DataFrame(dict(ImageID=ids))

In [104]:
pred_list = []
pred = pd.concat([id_df, pd.DataFrame(np.vstack(preds_soft))], axis=1)


In [114]:
from scipy import stats
from functools import reduce

In [108]:
pred_list.append(pred.drop('ImageID', axis=1))

0.3333333333333333

In [131]:
# geometric
num_ensembles = len(pred_list)
temp = reduce(lambda x, y: x*y, pred_list) ** (num_ensembles**(-1))
temp = temp.apply(lambda x: x.argmax(), axis=1)

In [138]:
# arithmetic
num_ensembles = len(pred_list)
temp = sum(pred_list) /num_ensembles
temp = temp.apply(lambda x: x.argmax(), axis=1)

In [149]:
len(pred_list)

3

In [199]:
pred_list = []

In [204]:
pred_list.append(pd.DataFrame(dict(ans=np.hstack(preds_hard))))

In [208]:
temp = pd.concat(pred_list, axis=1)

In [227]:
pd.Series([1,2,3,3,3,3,3,4]).value_counts().index[0]

3

In [228]:
temp.apply(lambda x: x.value_counts().index[0], axis=1)

0    355
1    193
2    355
3    193
4    355
5    193
6    355
7    193
8    355
9    193
dtype: int64

In [154]:
(np.array(weight_list) / sum(weight_list)).sum()

AttributeError: 'int' object has no attribute 'sum'

In [180]:
weights = map(lambda x: x / sum(weight_list), weight_list)
temp = sum([w*pred for w, pred in zip(weights, pred_list)])

In [187]:
temp.apply(lambda x: np.argmax(x), axis=1)

0    355
1    193
2    355
3    193
4    355
5    193
6    355
7    193
8    355
9    193
dtype: int64

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000708,0.000679,0.001170,0.000504,0.000462,0.000621,0.001066,0.001346,0.000776,0.000535,...,0.000872,0.001203,0.001407,0.001638,0.000848,0.000941,0.001244,0.000705,0.000730,0.000735
1,0.000729,0.000673,0.001306,0.000516,0.000364,0.000558,0.001022,0.001373,0.000785,0.000613,...,0.000938,0.001414,0.001361,0.001493,0.000758,0.001131,0.001192,0.000691,0.000791,0.000821
2,0.000708,0.000679,0.001170,0.000504,0.000462,0.000621,0.001066,0.001346,0.000776,0.000535,...,0.000872,0.001203,0.001407,0.001638,0.000848,0.000941,0.001244,0.000705,0.000730,0.000735
3,0.000729,0.000673,0.001306,0.000516,0.000364,0.000558,0.001022,0.001373,0.000785,0.000613,...,0.000938,0.001414,0.001361,0.001493,0.000758,0.001131,0.001192,0.000691,0.000791,0.000821
4,0.000708,0.000679,0.001170,0.000504,0.000462,0.000621,0.001066,0.001346,0.000776,0.000535,...,0.000872,0.001203,0.001407,0.001638,0.000848,0.000941,0.001244,0.000705,0.000730,0.000735
5,0.000729,0.000673,0.001306,0.000516,0.000364,0.000558,0.001022,0.001373,0.000785,0.000613,...,0.000938,0.001414,0.001361,0.001493,0.000758,0.001131,0.001192,0.000691,0.000791,0.000821
6,0.000708,0.000679,0.001170,0.000504,0.000462,0.000621,0.001066,0.001346,0.000776,0.000535,...,0.000872,0.001203,0.001407,0.001638,0.000848,0.000941,0.001244,0.000705,0.000730,0.000735
7,0.000729,0.000673,0.001306,0.000516,0.000364,0.000558,0.001022,0.001373,0.000785,0.000613,...,0.000938,0.001414,0.001361,0.001493,0.000758,0.001131,0.001192,0.000691,0.000791,0.000821
8,0.000708,0.000679,0.001170,0.000504,0.000462,0.000621,0.001066,0.001346,0.000776,0.000535,...,0.000872,0.001203,0.001407,0.001638,0.000848,0.000941,0.001244,0.000705,0.000730,0.000735
9,0.000729,0.000673,0.001306,0.000516,0.000364,0.000558,0.001022,0.001373,0.000785,0.000613,...,0.000938,0.001414,0.001361,0.001493,0.000758,0.001131,0.001192,0.000691,0.000791,0.000821
